In [1]:
# gpu 확인

import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
Tesla T4


# 데이터 불러오기


In [61]:
import pandas as pd
import os
import glob

# 엑셀 파일들이 저장된 디렉토리 경로
folder_path = 'C:\Users\r2com\Desktop\AI-STARS\labeled'

# 해당 디렉토리 내의 모든 .xlsx 파일 경로 가져오기
excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))

# 모든 엑셀 파일을 읽어서 리스트에 저장
df_list = [pd.read_excel(file) for file in excel_files]

# 하나의 DataFrame으로 병합
df = pd.concat(df_list, ignore_index=True)

In [62]:
df = df[['text', 'label']]

In [63]:
df['label'] = df['label'].apply(
    lambda x: int(x.split('_')[-1]) if isinstance(x, str) and x.startswith('LABEL_') else x
)

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3805 entries, 0 to 3804
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    3805 non-null   object
 1   label   3805 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 59.6+ KB


# 모델

In [9]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, AutoConfig

In [10]:
config = AutoConfig.from_pretrained(
    "alsgyu/sentiment-analysis-fine-tuned-model",
    num_labels=3,

    # 드롭아웃
    # hidden_dropout_prob=0.3,
    # attention_probs_dropout_prob=0.3
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

In [11]:
# 모델 로드
model = AutoModelForSequenceClassification.from_pretrained(
    "alsgyu/sentiment-analysis-fine-tuned-model",
    config=config
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained("alsgyu/sentiment-analysis-fine-tuned-model")

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/722k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [65]:
training_args = TrainingArguments(
    output_dir="C:\Users\r2com\Desktop\AI-STARS\results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5, #2e-5 -> 1e-5
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,           # 3->5
    weight_decay=0.01,
    save_total_limit=1,
    metric_for_best_model="f1",   # 성능 기준 : f1
    load_best_model_at_end=True,
    logging_dir="C:\Users\r2com\Desktop\AI-STARS\logs",
    greater_is_better=True
)

In [14]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00


In [15]:
def preprocess_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

In [66]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

df_train, df_valid = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=345)

train_dataset = Dataset.from_pandas(df_train)
valid_dataset = Dataset.from_pandas(df_valid)

# 확인
print("Training Dataset:", train_dataset)
print("Validation Dataset:", valid_dataset)

Training Dataset: Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 3044
})
Validation Dataset: Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 761
})


In [67]:
# 토큰화 적용
train_dataset = train_dataset.map(preprocess_function, batched=True)
valid_dataset = valid_dataset.map(preprocess_function, batched=True)

# 불필요한 열 제거
train_dataset = train_dataset.remove_columns(["text"])
valid_dataset = valid_dataset.remove_columns(["text"])

# 확인
print("Tokenized Training Dataset:", train_dataset)
print("Tokenized Validation Dataset:", valid_dataset)

Map:   0%|          | 0/3044 [00:00<?, ? examples/s]

Map:   0%|          | 0/761 [00:00<?, ? examples/s]

Tokenized Training Dataset: Dataset({
    features: ['label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3044
})
Tokenized Validation Dataset: Dataset({
    features: ['label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 761
})


In [68]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import EarlyStoppingCallback

# 평가 지표 함수
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# Trainer 정의
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)] # earlystopping callback 추가
)

<ipython-input-68-71a295e9c539>:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [69]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.198415,0.964520,0.964527,0.964564,0.964520
2,0.129500,0.227411,0.960578,0.960614,0.960801,0.960578
3,0.047400,0.253967,0.961892,0.961892,0.961892,0.961892


TrainOutput(global_step=1143, training_loss=0.08128958924861718, metrics={'train_runtime': 555.8256, 'train_samples_per_second': 27.383, 'train_steps_per_second': 3.427, 'total_flos': 1407862342202400.0, 'train_loss': 0.08128958924861718, 'epoch': 3.0})

# 모델 저장

In [71]:
# 모델 저장
trainer.model.save_pretrained("/content/drive/MyDrive/sentry/models/0416_model_epoch5_es2_lr1e-5_3800")

# 토크나이저 저장
tokenizer.save_pretrained("/content/drive/MyDrive/sentry/models/0416_tokenizer_epoch5_es2_lr1e-5_3800")

('/content/drive/MyDrive/sentry/models/0416_tokenizer_epoch5_es2_lr1e-5_3800/tokenizer_config.json',
 '/content/drive/MyDrive/sentry/models/0416_tokenizer_epoch5_es2_lr1e-5_3800/special_tokens_map.json',
 '/content/drive/MyDrive/sentry/models/0416_tokenizer_epoch5_es2_lr1e-5_3800/vocab.txt',
 '/content/drive/MyDrive/sentry/models/0416_tokenizer_epoch5_es2_lr1e-5_3800/added_tokens.json',
 '/content/drive/MyDrive/sentry/models/0416_tokenizer_epoch5_es2_lr1e-5_3800/tokenizer.json')